Implementazione dell'algoritmo per il trasferimento di stile descritto nell' articolo: [Image Style Transfer Using Convolutional Neural Networks](https://ieeexplore.ieee.org/document/7780634)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import vgg19

'''base_image_path = keras.utils.get_file("paris.jpg", "https://i.imgur.com/F28w3Ac.jpg")
style_reference_image_path = keras.utils.get_file(
    "starry_night.jpg", "https://i.imgur.com/9ooB60I.jpg"
)'''
# base_image_path rappresenta il contenuto dell'immagine che si vuole andare a sintetizzare
base_image_path = "/content/img1.jpg"

# style_reference_image_path rappresenta lo stile dell'immagine che si vuole sintetizzare
style_reference_image_path ="/content/sketch_cropped.png"

result_prefix = "result"

# vengono settati i pesi per le differenti funzioni di perdita
total_variation_weight = 1e-3
style_weight = 4e-0
content_weight = 4e-3

# Dimensione dell'immagine che si andrà a generare .
width, height = keras.preprocessing.image.load_img(base_image_path).size
img_nrows = 400
# vengono mantenute le proporzioni
img_ncols = int(width * img_nrows / height) 
print(base_image_path,img_nrows,img_ncols)

/content/img1.jpg 400 416


Prima di inserire le immagini all'interno della rete vgg19 è necessario che siano pre - processate e forzate ad avere una certa dimensione.

In [ ]:
def preprocess_image(image_path):    
    img = keras.preprocessing.image.load_img(
        image_path, target_size=(img_nrows, img_ncols)
    )
    img = keras.preprocessing.image.img_to_array(img)
    # viene aggiunta la dimensione per il batch
    img = np.expand_dims(img, axis=0) 
    img = vgg19.preprocess_input(img)
    return tf.convert_to_tensor(img)

#funzione che si occupa di eseguire i passi inversi della funzione 
def deprocess_image(x):
    # Util function to convert a tensor into a valid image
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype("uint8")
    return x



# Funzioni di perdita
Vengono definite le funzioni di perdita come descritto dall'articolo originario.

In [ ]:
# La matrice di gram viene impiegata per calcolare la funzione di perdita che riguarda lo stile: "style_loss"

def gram_matrix(x):
    x = tf.transpose(x, (2, 0, 1))
    features = tf.reshape(x, (tf.shape(x)[0], -1))
    gram = tf.matmul(features, tf.transpose(features))
    return gram


# Funzione di perdita per quanto riguarda lo stile:
# viene calcolata la matrice si gram in un particolare layer
# sia per l'immagine per cui si vuole imitare lo stile che per l'immagine
# che si sta andando a sintetizzare 

def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = np.size(S,0)
    #channels = 3
    size = img_nrows * img_ncols
    return tf.reduce_sum(tf.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))


# Funzione di perdita per calcolare lo scostamento tra il contenuto  
def content_loss(base, combination):
    return tf.reduce_sum(tf.square(combination - base))


# Funzione ausiliaria per ridurre il rumore
def total_variation_loss(x):
    a = tf.square(
        x[:, : img_nrows - 1, : img_ncols - 1, :] - x[:, 1:, : img_ncols - 1, :]
    )
    b = tf.square(
        x[:, : img_nrows - 1, : img_ncols - 1, :] - x[:, : img_nrows - 1, 1:, :]
    )
    return tf.reduce_sum(tf.pow(a + b, 1.25))



In [ ]:
import numpy as np
from PIL import Image

arr = np.random.rand(400,400,3) * 255
arr.clip(0,255).astype("uint8")
keras.preprocessing.image.save_img("p.jpg", arr.clip(0,255).astype("uint8"))


In [ ]:
# per estrarre il contenuto e lo stile di un'immagine
# viene impiegata la rete vgg19, include_top = False non include 
# gli ultimi livelli totalmente connessi
model = vgg19.VGG19(weights="imagenet", include_top=False)

# viene crato un dizionario in quanto quando verra' creato il modello
# si potra' recupare i layer necessari per ricostruire il contenuto e lo 
# stile 
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

feature_extractor = keras.Model(inputs=model.inputs, outputs=outputs_dict)


In [ ]:
# Lista di layer impiegati per calcolare la funzione di perdita dello stile
style_layer_names = [
    "block1_conv1",
    "block2_conv1",
    "block3_conv1",
    "block4_conv1",
    "block5_conv1",
]
# Layer impiegato per calcolare la perdita riguardo al contenuto .
content_layer_name = "block5_conv2"

In [ ]:
# funzione impiegata per calcolare la varie funzioni di perdita
# combination_image e' l'immagine di partenza che viene modellata per
# assumere lo stile dell'immagine "style_reference_image"
# e il contenuto di "base_image"
def compute_loss(combination_image, base_image, style_reference_image):
    input_tensor = tf.concat(
        [base_image, style_reference_image, combination_image], axis=0
    )
    #features e' un dizionario
    features = feature_extractor(input_tensor)

    
    loss = tf.zeros(shape=())

    # tramite il dizionario "features" viene estratto 
    # l'output del layer responsabile del contenuto 
    layer_features = features[content_layer_name]
    # il primo asse di layer_features indica il batch come definito 
    # dalla variabile input_tensor
    base_image_features = layer_features[0, :, :, :]
    combination_features = layer_features[2, :, :, :]
    loss = loss + content_weight * content_loss(
        base_image_features, combination_features
    )
     
    for layer_name in style_layer_names:
        layer_features = features[layer_name]
        style_reference_features = layer_features[1, :, :, :]
        combination_features = layer_features[2, :, :, :]
        sl = style_loss(style_reference_features, combination_features)
        loss += (style_weight / len(style_layer_names)) * sl

    # Si aggiunge anche una funzione di perdita per normalizzare l'immagine
    loss += total_variation_weight * total_variation_loss(combination_image)
    return loss



In [ ]:
#funzione impiegata per calcolare il gradiente della funzione 
# di perdita
@tf.function
def compute_loss_and_grads(combination_image, base_image, style_reference_image):
    with tf.GradientTape() as tape:
        loss = compute_loss(combination_image, base_image, style_reference_image)
    grads = tape.gradient(loss, combination_image)
    return loss, grads

In [ ]:
'''optimizer = keras.optimizers.SGD(
    keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1.0, decay_steps=4000, decay_rate=0.96
    )
)'''

optimizer = tf.optimizers.Adam(learning_rate=5, beta_1=0.99, epsilon=1e-1)

base_image = preprocess_image(base_image_path)
style_reference_image = preprocess_image(style_reference_image_path)
combination_image = tf.Variable(preprocess_image("/content/p.jpg"))
print("numero assi combination: ",combination_image.shape)
iterations = 50000
for i in range(1, iterations + 1):
    loss, grads = compute_loss_and_grads(
        combination_image, base_image, style_reference_image
    )
    print("numero assi grad: ",grads.shape)
    optimizer.apply_gradients([(grads, combination_image)])

    print("Iteration %d: loss=%.2f" % (i, loss))
    if i % 200 == 0:
        print("Iteration %d: loss=%.2f" % (i, loss))
        img = deprocess_image(combination_image.numpy())
        fname = result_prefix + "_at_iteration_%d.png" % i
        keras.preprocessing.image.save_img(fname, img)